In [1]:
import explicit_statetespace as efss
import ssr_kronecker_vector as ssr
import ssr_likelihood as ssr_lklhd
import likelihood as fss_lklhd
import kronecker_vector as fss
import numpy as np
import Utilityfunctions as utils

In [2]:
n = 2
sparsity = 0.5
log_theta = utils.random_theta(n=n, sparsity=sparsity)


In [3]:
state = np.random.randint(2, size=2*n+1)
length = 2**sum(state)
state


array([0, 1, 0, 0, 1])

In [4]:
mut = [state[j] + 2 * state[j + 1] for j in range(0, 2 * n, 2)]
mut.append(state[-1])
mut = np.array(mut)


In [20]:
# Test explicit ssr vs. shuffle trick ssr
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta) @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    ))


In [21]:
# Test no diag
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    q = ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    q[j] = 0
    assert (np.allclose(
        q,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, diag=False)
    ))


In [22]:
# Test diag
assert (np.allclose(
    np.diag(efss.ssr_build_q(dpoint=state, log_theta=log_theta)),
    ssr.kron_diag(log_theta=log_theta, n=n, state=state)
))


In [23]:
# Test R inverse
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        np.linalg.inv(np.identity(
            length) - efss.ssr_build_q(dpoint=state, log_theta=log_theta)) @ p,
        ssr_lklhd.R_i_inv_vec(log_theta=log_theta, x=p, lam=1, state=state),  
    ))


In [24]:
# Test transpose
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta).T @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, transpose=True)
    ))


In [5]:
# Test ssr vs fss
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    index = utils.ssr_to_fss(state)
    assert (np.allclose(
        efss.build_q(log_theta=log_theta)[np.ix_(index, index)] @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    ))

In [5]:
# Test log_likelihood

lam1, lam2 = np.random.random(2)
p = np.zeros(2**(2*n+1))
p[0] = 1
np.allclose(
    fss_lklhd.likelihood(theta=log_theta, pd=p, lam1=lam1, lam2=lam2),
    ssr_lklhd.log_likelihood(log_theta=log_theta, p_D=p, lam1=lam1, lam2=lam2, state=state))


c:\Users\Arbeit\Documents\MetaMHN\src\likelihood.py:45: RuntimeWarning: divide by zero encountered in log
  return pd.dot(np.nan_to_num(np.log(pTh)))
c:\Users\Arbeit\Documents\MetaMHN\src\ssr_likelihood.py:188: RuntimeWarning: divide by zero encountered in log
  np.nan_to_num(np.log(p_0))[reachable[utils.ssr_to_fss(state)]]


False

In [7]:
fss_lklhd.likelihood(theta=log_theta, pd=p, lam1=lam1, lam2=lam2)

-4.816402906700382

In [6]:
ssr_lklhd.log_likelihood(log_theta=log_theta, p_D=p, lam1=lam1, lam2=lam2, state=state)

0.0

In [18]:
p = np.zeros(2**(2*n + 1))
p[0] = 1
np.nonzero(fss_lklhd.jacobi(log_theta, p, lam=1))[0].shape

(20,)

In [7]:
fss_lklhd.jacobi(log_theta, p, lam=1)

array([5.37830423e-02, 0.00000000e+00, 0.00000000e+00, 1.75383041e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.96923740e-03, 0.00000000e+00, 0.00000000e+00, 3.71795349e-01,
       7.66091773e-04, 3.78134897e-04, 9.81476136e-04, 1.88437933e-02,
       1.65357021e-05, 5.11649177e-05, 6.94596135e-05, 1.49883396e-02,
       1.65357021e-05, 2.01404791e-05, 2.02287735e-04, 1.52899551e-02,
       4.48898063e-05, 2.95640832e-05, 9.66564650e-05, 3.44274305e-01])

In [8]:
ssr_lklhd.R_i_inv_vec(log_theta=log_theta, x=p, lam=1,
                      state=np.ones(2*n + 1, dtype=int))


ValueError: operands could not be broadcast together with shapes (32,) (4,) 